In [ ]:
# First, uninstall any existing installations of torch
!pip uninstall -y torch torchaudio torchvision

# Install the latest stable version of PyTorch with the appropriate CUDA version
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

In [ ]:

from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embedding_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [ ]:

import os
import pinecone
from google.colab import userdata

# API key from app.pinecone.io and environment from console
# The secrets are stored in colab secrets.
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = userdata.get('PINECONE_ENVIRONMENT')

pinecone.init(
    api_key=os.environ.get(PINECONE_API_KEY) or PINECONE_API_KEY,
    environment=os.environ.get(PINECONE_ENVIRONMENT) or PINECONE_ENVIRONMENT
)

In [ ]:

import time

index_name = 'nlp-chatbot'
createdIndex = False

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=384)
    createdIndex = True
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pinecone.Index(index_name)

In [ ]:
from langchain.vectorstores import Pinecone
# Define the field in metadata where the text will be stored
text_field = 'text'

# Initialize the vectorstore using LangChain Pinecone
vectorstore = Pinecone(index, embedding_model.embed_query, text_field)

In [ ]:

vectorstore.similarity_search(
    'Which NFL team represented the AFC at Super Bowl 50?',  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# The HF auth token is stored in colab serects and get it from there.
HF_AUTH =  userdata.get('HF_AUTH')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=HF_AUTH
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=HF_AUTH
)
model.eval()
print(f"Model loaded on {device}")

In [ ]:

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=HF_AUTH
)

In [ ]:

generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:

from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [ ]:
rag_pipeline('Which NFL team represented the AFC at Super Bowl 50?')

{'query': 'Which NFL team represented the AFC at Super Bowl 50?',
 'result': ' The Denver Broncos represented the AFC at Super Bowl 50.'}